<a href="https://colab.research.google.com/github/shanilev123456/Final-Project-ds/blob/master/data_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving rmi_data.xlsx to rmi_data.xlsx


In [3]:
import pandas as pd
df = pd.read_excel('rmi_data.xlsx')
df.head()

,Tender Number,Region,City,Purpose,Publication date,Status,Opening date,Submission deadline,Committee date,Compound Number,...,Winner Name,Final Price,Development Cost,Valuation Price,Minimum Price,Area m2,Plan,Parcel,Block,Section
0,198/2016,חיפה,"זכרון יעקב , מורדות הבאר",בנייה נמוכה/צמודת קרקע,25/09/2016,נדון בוועדת מכרזים,07/11/2016,26/12/2016 12:00,01/01/2017,68,...,"אלוש מישל, אלוש דנה, זלץ בן","2,150,101",NaN,"2,694,000","1,225,770",702,ש/619,68,11308.0,163.0
1,198/2016,חיפה,"זכרון יעקב , מורדות הבאר",בנייה נמוכה/צמודת קרקע,25/09/2016,נדון בוועדת מכרזים,07/11/2016,26/12/2016 12:00,01/01/2017,70,...,"פרץ סלומון, פרץ יפית, גיגי יהודית, גיגי שלמה","2,220,000",NaN,"2,694,000","1,225,770",701,ש/619,70,11308.0,165.0
2,198/2016,חיפה,"זכרון יעקב , מורדות הבאר",בנייה נמוכה/צמודת קרקע,25/09/2016,נדון בוועדת מכרזים,07/11/2016,26/12/2016 12:00,01/01/2017,71,...,"אורגל משה, ברכפלד אביגיל מרים, גלם אפרת, גלם רועי","1,880,800",NaN,"2,694,000","1,225,770",701,ש/619,71,11308.0,166.0
3,197/2016,ירושלים,"קרית גת , כרמי גת",בנייה נמוכה/צמודת קרקע,10/11/2016,נדון בוועדת מכרזים,10/11/2016,19/12/2016 12:00,27/12/2016,15,...,סבן ג'אן איב,"477,770","264,850","239,800","109,109",312,4/140/03/9,15,2909.0,22.0
4,197/2016,ירושלים,"קרית גת , כרמי גת",בנייה נמוכה/צמודת קרקע,10/11/2016,נדון בוועדת מכרזים,10/11/2016,19/12/2016 12:00,27/12/2016,A15,...,סבן ארי,"480,770","264,850","239,800","109,109",312,4/140/03/9,A15,2909.0,23.0


In [4]:

df = df.drop(columns=['Purpose', 'Status', 'Winner Name', 'Submission deadline', 'Plan'])

In [5]:
# הצגת נתונים חסרים לפני טיפול
print("missing data:")
print(df.isnull().sum())

# הסרת שורות שבהן 'Final Price' חסר או שווה ל-"אין הצעות למתחם זה"
df = df[df['Final Price'].notnull()]
df = df[df['Region'].notnull()]
df = df[df['Final Price'] != "אין הצעות למתחם זה"]

# המרת עמודות מספריות למספרים על ידי הסרת פסיקים והמרה ל-float
# רשימת העמודות שצריכות להיות מספריות
numeric_columns = ['Final Price', 'Valuation Price', 'Minimum Price', 'Development Cost']

for col in numeric_columns:
    # המרת העמודה למחרוזת (למקרה שיש ערכים מעורבים), הסרת פסיקים, והמרה למספר
    df[col] = df[col].astype(str).str.replace(',', '').astype(float)

# טיפול בנתונים חסרים בעמודות מספריות
df['Units'] = df['Units'].fillna(df['Units'].median())
df['Valuation Price'] = df['Valuation Price'].fillna(df['Valuation Price'].median())
df['Minimum Price'] = df['Minimum Price'].fillna(df['Minimum Price'].median())
df['Development Cost'] = df['Development Cost'].fillna(df['Development Cost'].mean())

# הסרת שורות שבהן 'Compound Number' חסר
df = df[df['Compound Number'].notnull()]
print(f"missing data Compound Number: {len(df)}")

# עמודות גיאוגרפיות - מילוי בערך ברירת מחדל
df['Parcel'] = df['Parcel'].fillna("לא קיים")
df['Block'] = df['Block'].fillna("לא קיים")
df['Section'] = df['Section'].fillna("לא קיים")

# הצגת נתונים חסרים לאחר טיפול
print("\nmissing data after:")
print(df.isnull().sum())
print(f"missing data len: {len(df)}")

missing data:
Tender Number        396
Region               396
City                 396
Publication date     396
Opening date         396
Committee date       396
Compound Number        6
Units                229
Final Price         3507
Development Cost    1402
Valuation Price      898
Minimum Price       2838
Area m2                2
Parcel               614
Block                263
Section              263
dtype: int64
missing data Compound Number: 7588

missing data after:
Tender Number       0
Region              0
City                0
Publication date    0
Opening date        0
Committee date      0
Compound Number     0
Units               0
Final Price         0
Development Cost    0
Valuation Price     0
Minimum Price       0
Area m2             0
Parcel              0
Block               0
Section             0
dtype: int64
missing data len: 7588


In [6]:

df['Final Price'] = df['Final Price'].replace(',', '', regex=True).astype(float)
df['Area m2'] = df['Area m2'].replace(',', '', regex=True).astype(float)
print(f"before filter Final Price: {len(df)}")
# גבולות ידניים נוספים
lower_bound = df['Final Price'].quantile(0.05)
upper_bound = df['Final Price'].quantile(0.95)
df = df[(df['Final Price'] >= lower_bound) & (df['Final Price'] <= upper_bound)]
print(f"after filter Final Price: {len(df)}")
df = df[(df['Area m2'] >= 50) & (df['Area m2'] <= 50000)]
print(f"after filter Area m2: {len(df)}")

before filter Final Price: 7588
after filter Final Price: 6828
after filter Area m2: 6828


In [7]:

duplicate_rows = df.duplicated(subset=['Tender Number', 'Compound Number'], keep=False)
print(f"len of doubles row: {duplicate_rows.sum()}")

df = df.drop_duplicates(subset=['Tender Number', 'Compound Number'], keep='first')
print(f"after: {len(df)}")

len of doubles row: 0
after: 6828


In [8]:

df['Publication date'] = pd.to_datetime(df['Publication date'], errors='coerce')
df['Opening date'] = pd.to_datetime(df['Opening date'], errors='coerce')
df['Committee date'] = pd.to_datetime(df['Committee date'], errors='coerce')


df['Publication_year'] = df['Publication date'].dt.year
df['Publication_month'] = df['Publication date'].dt.month
df['Publication_day'] = df['Publication date'].dt.day

df['Opening_year'] = df['Opening date'].dt.year
df['Opening_month'] = df['Opening date'].dt.month
df['Opening_day'] = df['Opening date'].dt.day

df['Committee_year'] = df['Committee date'].dt.year
df['Committee_month'] = df['Committee date'].dt.month
df['Committee_day'] = df['Committee date'].dt.day


df['Days_Publication_to_Opening'] = (df['Opening date'] - df['Publication date']).dt.days
df['Days_Publication_to_Committee'] = (df['Committee date'] - df['Publication date']).dt.days
df['Days_Opening_to_Committee'] = (df['Committee date'] - df['Opening date']).dt.days


df['Days_Publication_to_Opening'] = df['Days_Publication_to_Opening'].fillna(df['Days_Publication_to_Opening'].median())
df['Days_Publication_to_Committee'] = df['Days_Publication_to_Committee'].fillna(df['Days_Publication_to_Committee'].median())
df['Days_Opening_to_Committee'] = df['Days_Opening_to_Committee'].fillna(df['Days_Opening_to_Committee'].median())


for col in ['Publication_year', 'Publication_month', 'Publication_day',
            'Opening_year', 'Opening_month', 'Opening_day',
            'Committee_year', 'Committee_month', 'Committee_day']:
    df[col] = df[col].fillna(df[col].median())


df = df.drop(columns=['Publication date', 'Opening date', 'Committee date'])


print(df.head())

  Tender Number   Region                      City Compound Number  Units  \
0      198/2016     חיפה  זכרון יעקב , מורדות הבאר              68    2.0   
1      198/2016     חיפה  זכרון יעקב , מורדות הבאר              70    2.0   
2      198/2016     חיפה  זכרון יעקב , מורדות הבאר              71    2.0   
3      197/2016  ירושלים         קרית גת , כרמי גת              15    1.0   
4      197/2016  ירושלים         קרית גת , כרמי גת             A15    1.0   

   Final Price  Development Cost  Valuation Price  Minimum Price  Area m2  \
0    2150101.0     662029.676967        2694000.0      1225770.0    702.0   
1    2220000.0     662029.676967        2694000.0      1225770.0    701.0   
2    1880800.0     662029.676967        2694000.0      1225770.0    701.0   
3     477770.0     264850.000000         239800.0       109109.0    312.0   
4     480770.0     264850.000000         239800.0       109109.0    312.0   

   ... Publication_day Opening_year Opening_month  Opening_day  \
0  ...  

<ipython-input-8-69884f13699a>:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Publication date'] = pd.to_datetime(df['Publication date'], errors='coerce')


In [9]:
print(df['Final Price'].describe())

count    6.828000e+03
mean     6.031142e+05
std      5.428984e+05
min      3.666600e+04
25%      2.330000e+05
50%      4.400000e+05
75%      7.821955e+05
max      2.690180e+06
Name: Final Price, dtype: float64


In [10]:
df['Price per m2'] = df['Final Price'] / df['Area m2']

numeric_columns = [
    'Final Price', 'Development Cost', 'Valuation Price', 'Minimum Price',
    'Area m2', 'Units',
    'Publication_year', 'Publication_month', 'Publication_day',
    'Opening_year', 'Opening_month', 'Opening_day',
    'Committee_year', 'Committee_month', 'Committee_day',
    'Days_Publication_to_Opening', 'Days_Publication_to_Committee', 'Days_Opening_to_Committee','Price per m2'
]

for col in numeric_columns:
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())


print(df.head())

  Tender Number   Region                      City Compound Number     Units  \
0      198/2016     חיפה  זכרון יעקב , מורדות הבאר              68  0.015152   
1      198/2016     חיפה  זכרון יעקב , מורדות הבאר              70  0.015152   
2      198/2016     חיפה  זכרון יעקב , מורדות הבאר              71  0.015152   
3      197/2016  ירושלים         קרית גת , כרמי גת              15  0.000000   
4      197/2016  ירושלים         קרית גת , כרמי גת             A15  0.000000   

   Final Price  Development Cost  Valuation Price  Minimum Price   Area m2  \
0     0.796466          0.048428         0.419999       0.522931  0.035977   
1     0.822809          0.048428         0.419999       0.522931  0.035920   
2     0.694978          0.048428         0.419999       0.522931  0.035920   
3     0.166234          0.019146         0.034869       0.044437  0.013776   
4     0.167364          0.019146         0.034869       0.044437  0.013776   

   ... Opening_year Opening_month Opening_day  Com

In [11]:
print(df['Final Price'].describe())

count    6828.000000
mean        0.213471
std         0.204596
min         0.000000
25%         0.073990
50%         0.152000
75%         0.280959
max         1.000000
Name: Final Price, dtype: float64


In [12]:

df = pd.get_dummies(df, columns=['Region', 'City'], prefix=['Region', 'City'])


target_encode_columns = ['Block', 'Parcel', 'Section']

for col in target_encode_columns:
    target_mean = df.groupby(col)['Final Price'].mean()
    df[col] = df[col].map(target_mean)

for col in target_encode_columns:
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())

In [13]:
import pandas as pd


city_columns = [col for col in df.columns if col.startswith('City_')]


cities = set()
for col in city_columns:
    city_name = col.replace('City_', '').split(',')[0].strip()
    cities.add(city_name)


for city in cities:
    city_specific_columns = [col for col in city_columns if col.startswith(f'City_{city}')]
    df[f'City_{city}'] = df[city_specific_columns].any(axis=1)


city_avg_price_per_m2 = {}
for city in cities:
    avg_price = df[df[f'City_{city}']]['Price per m2'].mean()
    city_avg_price_per_m2[city] = avg_price


df['City_Avg_PricePerM2'] = 0.0
for city in cities:
    df.loc[df[f'City_{city}'], 'City_Avg_PricePerM2'] = city_avg_price_per_m2[city]



# ביצוע One-Hot Encoding לעמודה 'Tender Number'
tender_one_hot = pd.get_dummies(df['Tender Number'], prefix='Tender')

# הסרת העמודה 'Tender Number' מה-DataFrame המקורי
df = df.drop('Tender Number', axis=1)

# הוספת העמודות החדשות של One-Hot Encoding בסוף ה-DataFrame
df = pd.concat([df, tender_one_hot], axis=1)


<ipython-input-13-5b0d0e3e9371>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'City_{city}'] = df[city_specific_columns].any(axis=1)
<ipython-input-13-5b0d0e3e9371>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'City_{city}'] = df[city_specific_columns].any(axis=1)
<ipython-input-13-5b0d0e3e9371>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.

In [14]:
df.to_csv('cleaned_data.csv', index=False, encoding='utf-8-sig')
files.download('cleaned_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>